# Substitutional Point Defects in Graphene

## 0. Introduction

This tutorial demonstrates the process of creating materials with substitution defects, based on the work presented in the following manuscript, where nitrogen defects in graphene are studied.

[//]: # (<embed src="https://journals.aps.org/prb/abstract/10.1103/PhysRevB.84.245446" width="100%" height="300">)

> **Yoshitaka Fujimoto and Susumu Saito**, "Formation, stabilities, and electronic properties of nitrogen defects in graphene", Physical Review B, 2011. [DOI: 10.1103/PhysRevB.84.245446](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.84.245446).



## 1. Prepare the Environment
### 1.1. Set up defects parameters 
Defect Configuration parameters are described in [Defect Configuration](https://github.com/Exabyte-io/made/blob/main/src/py/mat3ra/made/tools/build/defect/configuration.py).

In [ ]:
CHEMICAL_ELEMENT = "N"  # Element to be placed at the site (ignored for vacancy)

SUPERCELL_MATRIX = [[4, 0, 0], [0, 4, 0], [0, 0, 1]]

DEFECT_CONFIGS = [
    {
        "defect_type": "substitution",
        "approximate_coordinate": [0.65, 0.25, 0.5],
        "chemical_element": CHEMICAL_ELEMENT,
    },
      {
        "defect_type": "substitution",
        "approximate_coordinate": [0.75, 0.55, 0.5],
        "chemical_element": CHEMICAL_ELEMENT,
    },
      {
        "defect_type": "substitution",
        "approximate_coordinate": [0.45, 0.25, 0.5],
        "chemical_element": CHEMICAL_ELEMENT,
    },
      {
        "defect_type": "vacancy",
        "approximate_coordinate": [0.51, 0.5, 0.5],
    },
    
]  

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages

    await install_packages("create_point_defect.ipynb", "../../config.yml")

### 1.3. Get input Graphene material

In [ ]:
from utils.jupyterlite import load_material_from_folder

material = load_material_from_folder("../uploads", "Graphene")

### 1.4. Create and preview Supercell

In [ ]:
from utils.visualize import visualize_materials as visualize
from mat3ra.made.tools.build.supercell import create_supercell

supercell = create_supercell(material, supercell_matrix=SUPERCELL_MATRIX)
visualize(supercell, repetitions=[1, 1, 1], rotation="0x")

## 2. Create the Defect
### 2.1. Set defect configurations and builder

In [ ]:
from mat3ra.made.tools.build.defect import PointDefectConfiguration
from mat3ra.made.tools.build.defect.builders import PointDefectBuilderParameters

configurations = []
for defect in DEFECT_CONFIGS:
    if "site_id" in defect:
        config = PointDefectConfiguration.from_site_id(crystal=supercell, **defect)
    elif "coordinate" in defect:
        config = PointDefectConfiguration(crystal=supercell, **defect)
    elif "approximate_coordinate" in defect:
        config = PointDefectConfiguration.from_approximate_position(crystal=supercell, **defect)
    else:
        raise ValueError(f"Invalid defect configuration: {defect}")
    configurations.append(config)

defect_configurations = configurations

defect_builder_parameters = PointDefectBuilderParameters(center_defect=False)

### 2.2. Create the defects

In [ ]:
from mat3ra.made.tools.build.defect import create_defects

material_with_defect = create_defects(builder_parameters=defect_builder_parameters, configurations=defect_configurations)

## 3. Visualize the Defects

In [ ]:
from utils.visualize import visualize_materials as visualize

visualize([{"material": supercell, "title": "Original material"},
           {"material": material_with_defect, "title": f"Material with defect"}],
          rotation="-90x")
visualize([{"material": supercell, "title": "Original material"},
           {"material": material_with_defect, "title": f"Material with defect"}])

## 4. Write resulting material to the folder

In [ ]:
from utils.jupyterlite import write_materials_to_folder

write_materials_to_folder([material_with_defect], "../uploads")